In [ ]:
from openai import OpenAI
import requests
import json
import base64
import os
import pandas as pd
import re
from pathlib import Path
import time

client = OpenAI(
    api_key = "",
    base_url = "https://api.moonshot.cn/v1",
)


# 从 Excel 文件中读取问题（假设文本数据在第二列）
imgs_questions = pd.read_csv("../Dataset/SoMDataset/Auxiliary_Relational/1_shape/1_shape.csv")
# 获取所有描述文本（假设文本在第二列）
text_list = imgs_questions.iloc[:, 1]  # 获取文本列
# 本地图片文件夹路径
folder_path = "../Dataset/SoMDataset/Auxiliary_Relational/1_shape/imgs"  # 替换为你的文件夹路径

# 单轮对话的消息结构
def create_message(file_content, text):
    return [
    {
        "role": "system",
        "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。",
    },
    {
        "role": "system",
        "content": file_content,
    },
    {"role": "user", "content": text},
]

# 获取文件夹中的所有图片
image_files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))],
    key=lambda x: int(re.match(r'(\d+)', x).group(1))
)

# 遍历文件夹中的所有图片文件和文本
for idx, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    text = text_list.iloc[idx]  # 从 text_list 获取当前图片对应的文本
    print(text)
    file_object = client.files.create(file=Path(image_path), purpose="file-extract")

    file_content = client.files.content(file_id=file_object.id).text
    print(file_content)
    # 构建消息
    messages = create_message(file_content, text)
    # 然后调用 chat-completion, 获取 Kimi 的回答
    completion = client.chat.completions.create(
    model="moonshot-v1-32k",
    messages=messages,
    temperature=0.3,
    )
    
    print(completion.choices[0].message)
    time.sleep(2)

